# NOTE
This has nothing to do with the camera derivative.
It tests the AD version primarily visible discontinuous derivatives.

In [1]:
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.sep.join(cwd_split[:cwd_split.index('mitsuba-ADjacobian')])
sys.path.insert(0, f"{root}/mitsuba-ADjacobian/build/python")

In [2]:
import drjit as dr
import mitsuba as mi
import os

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

mi.set_variant('cuda_ad_rgb')

# Scene

In [3]:
from mitsuba.scalar_rgb import Transform4f as T
integrator = {
    'type': 'direct_projective',
    'guiding': 'none',
    'sppp': 1,
    'sppi': 0
}

img_size = 100

sensor_to_world = T.look_at(
                        origin=(0, 0, 2),
                        target=(0, 0, 0),
                        up=(0, 1, 0)
                    )
original_trafo = T.translate([-0.4, 0, 1]).scale(0.4)

scene = mi.load_dict({
    'type': 'scene',
    'integrator': integrator,
    'sensor':  {
        'type': 'perspective',
        'to_world': sensor_to_world,
        'fov': 90,
        'film': {
            'type': 'hdrfilm',
            'width': img_size,
            'height': img_size,
            # 'rfilter': { 'type': 'gaussian' },
            # 'rfilter': {'type': 'tent', 'radius': 1},
            'rfilter': {'type': 'box'},
            'sample_border': True
        },
    },
    'wall': {
        'type': 'rectangle',
        'to_world': T.translate([0, 0, 0]).scale(2),
        'bsdf': {
            'type': 'diffuse',
            'reflectance': { 'type': 'rgb', 'value': (0.5, 0.5, 0.5) },
        },
        'emitter': {
            'type': 'area',
            'radiance': {'type': 'rgb', 'value': [1, 0, 0]}
        },
    },
    'block': {
        # 'type': 'disk',
        'type': 'sphere',
        # 'type': 'rectangle',
        'to_world': original_trafo,
        'bsdf': {
            'type': 'diffuse'
        },
        'emitter': {
            'type': 'area',
            'radiance': {'type': 'rgb', 'value': [0, 0, 1]}
        },
    },
    # 'light': {
    #     'type': 'disk',
    #     'emitter': {
    #         'type': 'area',
    #         'radiance': {'type': 'rgb', 'value': [10, 10, 10]}
    #     },
    #     'to_world': T.look_at(
    #                     origin=(1, 1, 3),
    #                     target=(0, 0, 0),
    #                     up=(0, 1, 0)
    #                 ).scale(0.5),
    # }
})

In [4]:
params = mi.traverse(scene)
key = 'block.to_world'

def apply_trafo(params, value):
    trafo = mi.Transform4f.translate([value, 0.0, 0.0])
    # trafo = mi.Transform4f.translate([0.0, 0.0, 0.0]).rotate([0, 0, 1], value)
    # trafo = mi.Transform4f.translate([0.0, 0.0, value])

    params[key] = trafo @ original_trafo
    params.update()

In [5]:
img_ref = mi.render(scene, seed=0, spp=1024)
mi.Bitmap(img_ref).write("output/primal.exr")

# Gradient

In [6]:
# Finite differences
def fd(epsilon=1e-3):
    fd_spp = 512
    fd_repeat = 16

    img1 = dr.zeros(mi.TensorXf, (img_size, img_size, 3))
    img2 = dr.zeros(mi.TensorXf, (img_size, img_size, 3))

    for it in range(fd_repeat):
        apply_trafo(params, -epsilon)
        img1 += mi.render(scene, spp=fd_spp, seed=it)
        params.update()

        apply_trafo(params, +epsilon)
        img2 += mi.render(scene, spp=fd_spp, seed=it)
        params.update()

        print(f"{it+1}/{fd_repeat}", end='\r')

    img_fd = (img2 - img1) / (epsilon*2) / fd_repeat
    # mi.Bitmap(img1 / fd_repeat).write("output/img1.exr")
    # mi.Bitmap(img2 / fd_repeat).write("output/img2.exr")

    return img_fd

if True:
    epsilon = 5e-4
    mi.Bitmap(fd(epsilon)).write("output/grad_fd_%.0e.exr" % epsilon)

    epsilon = 1e-3
    mi.Bitmap(fd(epsilon)).write("output/grad_fd_%.0e.exr" % epsilon)

# Boundary sample

In [7]:
x = mi.Float(0.0)
dr.enable_grad(x)
apply_trafo(params, x)
dr.forward(x, dr.ADFlag.ClearEdges)

image = mi.render(scene, params = params, spp=1024)

In [8]:
img_guided = dr.forward_to(image)
mi.Bitmap(img_guided).write("output/grad.exr")

J = [0.62864566 0.5893525  0.3077087  0.25282583 0.27613705 0.4433207
 0.25185633 0.38872793 0.65612185 0.26143536 0.65677226 0.2854114
 0.6581517  0.57298094 0.38997972 0.36390507 0.657116   0.25784633
 0.251079   0.38299975 0.60559    0.6400774  0.26260576 0.37100923
 0.4525558  0.5397115  0.47746295 0.6463558  0.3494778  0.25000706
 0.656369   0.53892267 0.46158892 0.25781158 0.26863596 0.63479525
 0.30057678 0.3161792  0.65596604 0.25000578 0.31092548 0.6206913
 0.3890466  0.3727904  0.38685915 0.52068186 0.2594773  0.32466695
 0.41065055 0.26282018 0.25740623 0.54286814 0.25705132 0.6550283
 0.37034583 0.26093337 0.2997377  0.3614077  0.63774884 0.3961431
 0.34004024 0.29251304 0.27874732 0.5674615  0.26106587 0.64998376
 0.61154234 0.32855406 0.4758131  0.25251317 0.56440204 0.25426444
 0.2997741  0.4184369  0.5973702  0.26379946 0.27945697 0.25543705
 0.42026386 0.4612712  0.65606284 0.5079735  0.36546296 0.38931686
 0.6474632  0.32292584 0.25703087 0.37586188 0.37100565 0.40540